In [ ]:
import pygame

# Global constants

# Colors in RGB
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)

## Your colors here
LIGHT_BLUE = (60, 220, 225)

## End of your colors

# Screen Dimensions
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

""" Player Class """
class Player(pygame.sprite.Sprite):
    
    # ------ Global Variables --------
    # Set Player width and height - CONSTANT VARIABLES
    PLAYER_WIDTH = 64
    PLAYER_HEIGHT = 64

    # Set speed vector
    playerX_change = 0
    playerY_change = 0
    # -------------------------------
    
 
    def __init__(self, playerX, playerY):
        """ Constructor function """
        # Call the parent's constructor
        super().__init__()
        
        # Set height, width
        self.playerImg = pygame.image.load("player.png")

        self.rect = self.playerImg.get_rect()
        self.rect.x = playerX
        self.rect.y = playerY
        
        # List of sprites we can bump against
        self.platform_list = None
        
    # This sets how much player will be moved when update() is called
    def move(self, direction, change):
        if (direction == "UP" or direction == "DOWN"):
            self.playerY_change = change
        if (direction == "RIGHT" or direction == "LEFT"):
            self.playerX_change = change
    
    # This stops player from moving - doesnt change player coord when update() is called
    def stop(self, direction):
        if (direction == "UP" or direction == "DOWN"):
            self.playerY_change = 0
        if (direction == "RIGHT" or direction == "LEFT"):
            self.playerX_change = 0
            
    def gravity(self):
        """ Calculate effect of gravity. """
        if self.playerY_change == 0:
            self.playerY_change = 1
        else:
            self.playerY_change += .35
 
        # See if we are on the ground.
        if self.rect.y >= SCREEN_HEIGHT - self.rect.height and self.playerY_change >= 0:
            self.playerY_change = 0
            self.rect.y = SCREEN_HEIGHT - self.rect.height
 
    def jump(self):
        """ Called when user hits 'jump' button. """
 
        # move down a bit and see if there is a platform below us.
        # Move down 2 pixels because it doesn't work well if we only move down
        # 1 when working with a platform moving down.
        self.rect.y += 2
        hitList = pygame.sprite.spritecollide(self, self.platform_list, False)
        self.rect.y -= 2
 
        # If it is ok to jump, set our speed upwards
        if len(hitList) > 0 or self.rect.bottom >= SCREEN_HEIGHT:
            self.playerY_change = -10
            
    # This updates player's position
    def update(self, screenWidth, screenHeight):
       # Gravity
        self.gravity()
 
        # see if moving in x-direction gives collision
        self.rect.x += self.playerX_change
        hitList = pygame.sprite.spritecollide(self, self.platform_list, False)
        for platform in hitList:
            if self.playerX_change > 0:
                self.rect.right = platform.rect.left
            elif self.playerX_change < 0:
                self.rect.left = platform.rect.right

        # see if moving in y-direction gives collision
        self.rect.y += self.playerY_change
        hitList = pygame.sprite.spritecollide(self, self.platform_list, False)
        for platform in hitList:
            if self.playerY_change > 0:
                self.rect.bottom = platform.rect.top
            elif self.playerY_change < 0:
                self.rect.top = platform.rect.bottom
                
            self.playerY_change = 0
                
        
        # Minimum and maximum boundery
        if self.rect.x < 0: # player went out of left border
            self.rect.x = 0
        if self.rect.x > screenWidth - self.PLAYER_WIDTH: # player went out of right border
            self.rect.x = screenWidth - self.PLAYER_WIDTH
            
        if self.rect.y < 0: # player went out of the top border
            self.rect.y = 0
        if self.rect.y > screenHeight - self.PLAYER_HEIGHT: # player went out of bottom border
            self.rect.y = screenHeight - self.PLAYER_HEIGHT
    
    def draw(self, screen):
        screen.blit(self.playerImg, self.rect)

        
# Define Platform Class
class Platform(pygame.sprite.Sprite):
    """ Platform the user can jump on """
 
    def __init__(self, width, height):
        
        super().__init__()
 
        self.image = pygame.Surface([width, height])
        self.image.fill(GREEN)
        
        self.rect = self.image.get_rect()

""" Main Program"""
def main():
    # Initialize the pygame instance
    pygame.init()
    
    # Set the height and width of the screen
    size = [SCREEN_WIDTH, SCREEN_HEIGHT]
    screen = pygame.display.set_mode(size)
    
    # Title of the game
    pygame.display.set_caption("HUMAN JUMPER")
    
    # Icon
    icon = pygame.image.load("icon.png")
    pygame.display.set_icon(icon)
    
    # Player
    player = Player(370, 480) # this is initial player position
    
    # Platforms
    platformList = pygame.sprite.Group()
    
    # Platform1
    platform = Platform(210, 70) # Platform(width, height)
    platform.rect.x = 500 # x-position of x-y coordinate
    platform.rect.y = 500 # y-position of x-y coordinate
    
    platformList.add(platform)
    
    # Platform2
    platform2 = Platform(210, 70) # Platform(width, height)
    platform2.rect.x = 100 # x-position of x-y coordinate
    platform2.rect.y = 500 # y-position of x-y coordinate
    
    platformList.add(platform2)
    
    # Assigning created platforms to player
    player.platform_list = platformList
        
    # Ending conditions
    done = False
    
    # Clock used to manage how fast the screen updates
    clock = pygame.time.Clock()
    
    # ---- Game Loop ----
    while not done:
        # Iterate through the pygame events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True
                
            # Detect key presses
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_RIGHT:
                    player.move("RIGHT", 2)
                if event.key == pygame.K_LEFT:
                    player.move("LEFT", -2)
                if event.key == pygame.K_UP:
                    player.jump()
                if event.key == pygame.K_DOWN:
                    player.move("DOWN", 2)
                
            if event.type == pygame.KEYUP:
                if event.key == pygame.K_RIGHT:
                    player.stop("RIGHT")
                if event.key == pygame.K_LEFT:
                    player.stop("LEFT")
                if event.key == pygame.K_DOWN:
                    player.stop("DOWN")
        
        ## -------- TO DO ----------------

        # Update player location/coordinate
        player.update(SCREEN_WIDTH, SCREEN_HEIGHT)
        
        # Change background color
        screen.fill(LIGHT_BLUE)
        
        # Draw the Player !! make sure to draw after screen
        platformList.draw(screen)
        player.draw(screen)
        
        ## ------ End of your code -------
        
        
        """ ==== DO NOT MODIFY ANY CODE BELOW THIS LINE === """

        # Limit to 60 frames per second
        clock.tick(60)
        
        # Update the screen
        pygame.display.update()
        
    # Exit
    pygame.quit()
    exit()
    
if __name__ == "__main__":
    main()

In [ ]:

# Define Moving Platform Class
class MovingPlatform(Platform):
    """ This is a fancier platform that can actually move. """
    change_x = 0
    change_y = 0
 
    boundary_top = 0
    boundary_bottom = 0
    boundary_left = 0
    boundary_right = 0
 
    player = None
 
    level = None
 
    def update(self):
        """ Move the platform.
            If the player is in the way, it will shove the player
            out of the way. This does NOT handle what happens if a
            platform shoves a player into another object. Make sure
            moving platforms have clearance to push the player around
            or add code to handle what happens if they don't. """
 
        # Move left/right
        self.rect.x += self.change_x
 
        # See if we hit the player
        hit = pygame.sprite.collide_rect(self, self.player)
        if hit:
            # We did hit the player. Shove the player around and
            # assume he/she won't hit anything else.
 
            # If we are moving right, set our right side
            # to the left side of the item we hit
            if self.change_x < 0:
                self.player.rect.right = self.rect.left
            else:
                # Otherwise if we are moving left, do the opposite.
                self.player.rect.left = self.rect.right
 
        # Move up/down
        self.rect.y += self.change_y
 
        # Check and see if we the player
        hit = pygame.sprite.collide_rect(self, self.player)
        if hit:
            # We did hit the player. Shove the player around and
            # assume he/she won't hit anything else.
 
            # Reset our position based on the top/bottom of the object.
            if self.change_y < 0:
                self.player.rect.bottom = self.rect.top
            else:
                self.player.rect.top = self.rect.bottom
 
        # Check the boundaries and see if we need to reverse
        # direction.
        if self.rect.bottom > self.boundary_bottom or self.rect.top < self.boundary_top:
            self.change_y *= -1
 
        cur_pos = self.rect.x - self.level.world_shift
        if cur_pos < self.boundary_left or cur_pos > self.boundary_right:
            self.change_x *= -1
            
# Define Level Class
class Level(object):
    """ This is a generic super-class used to define a level.
        Create a child class for each level with level-specific
        info. """
 
    def __init__(self, player):
        """ Constructor. Pass in a handle to player. Needed for when moving
            platforms collide with the player. """
        self.platform_list = pygame.sprite.Group()
        self.enemy_list = pygame.sprite.Group()
        self.player = player
         
        # Background image
        self.background = None
     
        # How far this world has been scrolled left/right
        self.world_shift = 0
        self.level_limit = -1000
 
    # Update everythign on this level
    def update(self):
        """ Update everything in this level."""
        self.platform_list.update()
        self.enemy_list.update()
 
    def draw(self, screen):
        """ Draw everything on this level. """
 
        # Draw the background
        screen.fill(BLUE)
 
        # Draw all the sprite lists that we have
        self.platform_list.draw(screen)
        self.enemy_list.draw(screen)
 
    def shift_world(self, shift_x):
        """ When the user moves left/right and we need to scroll everything:
        """
 
        # Keep track of the shift amount
        self.world_shift += shift_x
 
        # Go through all the sprite lists and shift
        for platform in self.platform_list:
            platform.rect.x += shift_x
 
        for enemy in self.enemy_list:
            enemy.rect.x += shift_x